### Reformat docRef_ids file

In [1]:
import re

pattern = re.compile(r'"(\d*)"')

i = 0
with open("docRef_ids_clean", "w+") as wf:
    with open("docRef_ids", 'r') as rf:
        for line in rf:
            match = pattern.search(line)
            if match:
                wf.writelines([match.group(1) + '\n'])
                i += 1
                if i % 10000000 == 0:
                    print(i)
print(f"Finished! Inserted {i} lines")

10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
100000000
Finished! Inserted 102563617 lines


### Create data structure

In [1]:
from __future__ import annotations

import bisect
import zlib
import sys


class Node:
    def __init__(self, value=None):
        self.value = value
        self.children = {}
        self.children_leaf = []

    def add_child(self, value):
        i = value[0]
        l = len(value)
        if l == 1:
            bisect.insort(self.children_leaf, i)
        elif not i in self.children:
            self.children[i] = Node(i)
        if l > 1:
            self.children[i].add_child(value[1:])

    def __str__(self, level=0):
        result = " " * level + '|-' + str(self.value) + '\n'
        for e in self.children_leaf:
            result += " " * (level + 1) + '#-' + str(e) + '\n'
        for k, node in self.children.items():
            result += node.__str__(level + 1)
        return result

    def size(self):
        return sys.getsizeof(self) + sum([c.size() for c in self.children.values()])

    def count(self):
        return 1 + sum([c.count() for c in self.children.values()])

    def to_id_list(self):
        result = []
        for cl in self.children_leaf:
            result.append((self.value if self.value else '') + cl)
        for k, v in self.children.items():
            result += [(self.value if self.value else '') + e for e in v.to_id_list()]
        return result
    
    def to_string_id_list(self, separator=','):
        result = ''
        for cl in self.children_leaf:
            result += (self.value if self.value else '') + cl + separator
        for k, v in self.children.items():
            for e in v.to_id_list():
                result += (self.value if self.value else '') + e + separator
        return result        

    def to_string(self):
        result = str(self.value) if self.value else ''
        tmp = ','.join(list(self.children_leaf) + [v.to_string() for v in self.children.values()])
        if tmp:
            result = result + '(' + tmp + ')'
        return result

    def from_string(self, string: str, string_len, pos=0):
        if string_len == pos:
            return pos
        number = string[pos + 1]
        following_char = string[pos + 2]
        if following_char == ')':
            self.children_leaf.append(number)
            return pos + 2

        if following_char == ',':
            self.children_leaf.append(number)
            return self.from_string(string, string_len, pos + 2)

        if following_char == '(':
            node = Node(number)
            self.children[number] = node
            tmp_pos = node.from_string(string, string_len, pos + 2)
            if tmp_pos < string_len:
                if string[tmp_pos+1] == ',':
                    return self.from_string(string, string_len, tmp_pos+1)
                return tmp_pos

    def to_compressed_string(self, encoding='utf-8') -> bytes:
        return zlib.compress(self.to_string().encode(encoding))

    def from_compressed_string(string: bytes, encoding='utf-8') -> Node:
        tmp = zlib.decompress(string).decode(encoding)
        tmp_len = len(tmp)
        node = Node()
        node.from_string(tmp, tmp_len)
        return node

#### Some tests on the tree data structure

In [2]:
tree = Node()
tree.add_child("123456")
tree.add_child("12346")
tree.add_child("12349")
tree.add_child("12348")
tree.add_child("12345")
tree.add_child("12343")
tree.add_child("123437")
tree.add_child("12341")
tree.add_child("12")
print(tree)
print(tree.to_id_list())
print("Tree repr: " + tree.to_string())
print("Repr size=" + str(sys.getsizeof(tree.to_string())))
print("Compressed repr size=" + str(sys.getsizeof(tree.to_compressed_string())))
tree = Node.from_compressed_string(tree.to_compressed_string())
print(tree.to_string())
ids = tree.to_id_list()
other_tree = Node()
for id in ids:
    other_tree.add_child(id)
print(ids == other_tree.to_id_list())
print(tree.to_string_id_list())

|-None
 |-1
  #-2
  |-2
   |-3
    |-4
     #-1
     #-3
     #-5
     #-6
     #-8
     #-9
     |-5
      #-6
     |-3
      #-7

['12', '12341', '12343', '12345', '12346', '12348', '12349', '123456', '123437']
Tree repr: (1(2,2(3(4(1,3,5,6,8,9,5(6),3(7))))))
Repr size=86
Compressed repr size=74
(1(2,2(3(4(1,3,5,6,8,9,5(6),3(7))))))
True
12,12341,12343,12345,12346,12348,12349,123456,123437,


### Import 10 millions ids in the tree

In [3]:
tree = Node()
i = 0
tmp = [] # for comparision store in a list

from datetime import datetime, timedelta

add_child_time = timedelta(0)
with open("docRef_ids_clean", "r") as f:
    for line in f:
        a = datetime.now()
        tree.add_child(line[:-1])
        add_child_time += datetime.now()-a
        tmp.append(line[:-1])
        i += 1
        if i % 10000000 == 0:
            break

In [4]:
print(f"""
Times :
  * add_child_time={add_child_time.total_seconds()}s (mean={add_child_time.total_seconds()/i}s per added id)
""")


Times :
  * add_child_time=74.364375s (mean=7.4364375e-06s per added id)



### export and import from string/compressed string

In [6]:
a = datetime.now()
to_string = tree.to_string()
to_string_time = datetime.now()-a

a = datetime.now()
an_other_tree = Node()
from_string = an_other_tree.from_string(to_string, len(to_string))
from_string_time = datetime.now()-a

a = datetime.now()
to_compressed_string = tree.to_compressed_string()
to_compressed_string_time = datetime.now()-a

a = datetime.now()
from_compressed_string = Node.from_compressed_string(to_compressed_string)
from_compressed_string_time = datetime.now()-a

a = datetime.now()
to_id_list = tree.to_id_list()
to_id_list_time = datetime.now()-a

a = datetime.now()
to_string_id_list = tree.to_string_id_list()
to_string_id_list_time = datetime.now()-a

print(f"""
Times :
  * to_string_time={to_string_time.total_seconds()}s (mean={to_string_time.total_seconds()/i}s)
  * from_string_time={from_string_time.total_seconds()}s (mean={from_string_time.total_seconds()/i}s)
  * to_compressed_string_time={to_compressed_string_time.total_seconds()}s (mean={to_compressed_string_time.total_seconds()/i}s)
  * from_compressed_string_time={from_compressed_string_time.total_seconds()}s (mean={from_compressed_string_time.total_seconds()/i}s)
  * to_id_list_time={to_id_list_time.total_seconds()}s (mean={to_id_list_time.total_seconds()/i}s)
  * to_string_id_list_time={to_string_id_list_time.total_seconds()}s (mean={to_string_id_list_time.total_seconds()/i}s)
""")
            
            
print(f"""
For {i} IDs :
  * len(list)={len(tmp)}
  * len(id_list)={len(to_id_list)}
  * list == id_list = {sorted(tmp) == sorted(to_id_list)}
  * sizeof(list)={sys.getsizeof(tmp)}
  * sizeof(id_list)={sys.getsizeof(to_id_list)}
  * tree.count()={tree.count()}
  * tree.size()={tree.size()}
  * sizeof(tree.to_string())={sys.getsizeof(to_string)}
  * sizeof(tree.to_compressed_string())={sys.getsizeof(to_compressed_string)} (ratio compared to list={int(sys.getsizeof(tmp)/sys.getsizeof(to_compressed_string))}x smaller)""")


Times :
  * to_string_time=4.833165s (mean=4.833165e-07s)
  * from_string_time=0.000193s (mean=1.93e-11s)
  * to_compressed_string_time=5.808068s (mean=5.808067999999999e-07s)
  * from_compressed_string_time=0.079834s (mean=7.9834e-09s)
  * to_id_list_time=18.700069s (mean=1.8700069e-06s)
  * to_string_id_list_time=20.862333s (mean=2.0862333e-06s)


For 10000000 IDs :
  * len(list)=10000000
  * len(id_list)=10000000
  * list == id_list = True
  * sizeof(list)=81528064
  * sizeof(id_list)=90000120
  * tree.count()=3288635
  * tree.size()=210472640
  * sizeof(tree.to_string())=29865952
  * sizeof(tree.to_compressed_string())=3891021 (ratio compared to list=20x smaller)
